In [43]:
import os
import codecs
import pickle
import numpy as np

def save(filename, data):
    with open(filename, 'wb') as output:
        pickle.dump(data, output)

def load(filename):
    with open(filename, 'rb') as output:
        data = pickle.load(output)
    return data

filename1 = 'part_whole_dataset.txt'
filename2 = 'instances_order.pkl'
filename3 = 'co_instance_pattern.pkl'
filename4 = 'tuples.pkl'
filename5 = 'patterns_order.pkl'

path1 = os.path.join(os.getcwd(), filename1)
path2 = os.path.join(os.getcwd(), filename2)
path3 = os.path.join(os.getcwd(), filename3)
path4 = os.path.join(os.getcwd(), filename4)
path5 = os.path.join(os.getcwd(), filename5)

entities_pattern = []    # 存储三元组
with codecs.open(path1, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f.readlines():
        tmp = ''
        cnt = 0
        entity = []
        for strs in line:
            tmp += strs
            if strs == '\t':
                cnt += 1
                if cnt <= 3:
                    tmp = ''
                else:
                    entity.append(tmp.replace('\t', ''))
                    tmp =''
                if cnt >= 6:
                    entities_pattern.append(entity)
                    break


instances_order = {}
patterns_order = {}
tuples = {}
cnt1 = 0
cnt2 = 0
for term in entities_pattern:
    instance = (term[0], term[2])
    if instance not in instances_order:
        instances_order[instance] = cnt1
        cnt1 += 1
    
    if term[1] not in patterns_order: 
        patterns_order[term[1]] = cnt2
        cnt2 += 1
    tuples[instance] = term[1]
    
    

# 共线矩阵
co_instance_pattern = np.zeros(shape=(len(instances_order), len(patterns_order)))
for term in entities_pattern:
    x = instances_order[(term[0], term[2])]
    y = patterns_order[term[1]]
    co_instance_pattern[x][y] += 1
    

# print(entities_pattern)
# print('---')
# print(instances_order)
# print('-----')
# print(patterns_order)
# print('-----')
# print(co_instance_pattern)

save(path2, instances_order)
save(path3, co_instance_pattern)
save(path4, tuples)
save(path5, patterns_order)


In [2]:
import os
import codecs
import pickle

def save(filename, data):
    with open(filename, 'wb') as output:
        pickle.dump(data, output)
        
filename1 = 'glove.6B.100d.txt'
filename2 = 'embedding_dict.pkl'

path1 = os.path.join('/home/mc/Jupyter/', filename1)
path2 = os.path.join(os.getcwd(), filename2)

dicts = {}
with codecs.open(path1, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        try:
            lists = line.strip().split()
            embedding = [float(x) for x in lists[-100:]]  # 从后往前数300维
            dicts[''.join(lists[:-100])] = embedding
        except:
            print(lists)
            pass

save(path2, dicts)
        


In [28]:
# 基于k-means的方法

import numpy as np
from sklearn.cluster import KMeans

def load(filename):
    with open(filename, 'rb') as output:
        data = pickle.load(output)
    return data

filename1 = 'embedding_dict.pkl'
filename2 = 'instances_order.pkl'
filename3 = 'tuples.pkl'

path1 = os.path.join(os.getcwd(), filename1)
path2 = os.path.join(os.getcwd(), filename2)
path3 = os.path.join(os.getcwd(), filename3)

embedding_dict = load(path1)
instances_order = load(path2)
tuples = load(path3)

offset_list = []
for instance in instances_order:
    x1, x2 = np.zeros(100), np.zeros(100)
    # 如果表示instace[0]的单词不止一个，比如'five member'
    if len(instance[0].split()) > 1:
        for term in instance[0].split():
            x1 += embedding_dict.get(term, np.zeros(100))    # 如果没有在embedding空间中出现，就返回0.
        x1 /= len(instance[0].split())
    else:
        x1 = embedding_dict.get(instance[0], np.zeros(100))
        
    if len(instance[1].split()) > 1:
        for term in instance[1].split():
            x2 += embedding_dict.get(term, np.zeros(100))
        x2 /= len(instance[1].split())
    else:
        x2 = embedding_dict.get(instance[1], np.zeros(100))
        
    offset_list.append([abs(i-j) for i, j in zip(x1, x2)]) # 对应元素减

k = 5
X = np.array(offset_list)
kmeans = KMeans(n_clusters = k).fit(X)

labels = kmeans.labels_
clusters_cents = kmeans.cluster_centers_

clusters_dict = {}   # key是类别i，value是一个属于类别i的元素组成的列表
for i in range(len(labels)):
    if clusters_dict.get(labels[i]) == None:
        clusters_dict[labels[i]] = [i,]
    else:
        clusters_dict[labels[i]].append(i)


selected_instances = []
order_instances = {v:k for k,v in instances_order.items()}   # 反转原来的字典

for i in range(k):
    minDis = float('inf')  # 初始化为最大值
    flag = -1
    for j in clusters_dict[i]:
        # 计算类别i的形心与所属类别i的每个点的欧式距离
        distance = np.sqrt(np.sum(np.square(X[j] - clusters_cents[i])))
        if minDis > distance:
            minDis = distance
            flag = j
    selected_instances.append(order_instances[flag])

print(selected_instances)
for term in selected_instances:
    print(tuples[term])
    

[('calcium', 'healthy bone and tooth'), ('exercise', 'a weight loss program'), ('breakfast', 'bread and cracker'), ('the board of directors', 'an executive committee'), ('the fifa headquarters', 'zrich')]
be necessary for
be a valuable element of
consist of
shall consist of
be locate in


In [29]:
# 基于NMF的方法 

import os
import numpy as np
from sklearn.decomposition import NMF

def load(filename):
    with open(filename, 'rb') as output:
        data = pickle.load(output)
    return data

filename1 = 'co_instance_pattern.pkl'
filename2 = 'instances_order.pkl'
filename3 = 'tuples.pkl'

path1 = os.path.join(os.getcwd(), filename1)
path2 = os.path.join(os.getcwd(), filename2)
path3 = os.path.join(os.getcwd(), filename3)

co_instance_pattern = load(path1)
instances_order = load(path2)
tuples = load(path3)

order_instances = {v:k for k,v in instances_order.items()}   # 反转原来的字典

k = 5
model = NMF(n_components=k, alpha=0.01)
W = model.fit_transform(co_instance_pattern)
H = model.components_

selected_instances = []
idx = np.argmax(W, axis=0)   # 获得每一列的最大值所在的行索引
for i in idx:
    selected_instances.append(order_instances[i]) 

print(selected_instances)
for term in selected_instances:
    print(tuples[term])


[('treatment', 'supportive care'), ('water', 'life'), ('the executive committee', 'the president'), ('protein', 'amino acid'), ('death', 'life')]
will involve
be a necessity for
consist of
be make up of
be the best part of


In [31]:
# 基于SVD的方法

import os
import numpy as np
from sklearn.decomposition import TruncatedSVD

def load(filename):
    with open(filename, 'rb') as output:
        data = pickle.load(output)
    return data

filename1 = 'co_instance_pattern.pkl' 
filename2 = 'instances_order.pkl'
filename3 = 'tuples.pkl'

path1 = os.path.join(os.getcwd(), filename1)
path2 = os.path.join(os.getcwd(), filename2)
path3 = os.path.join(os.getcwd(), filename3)

co_instance_pattern = load(path1)
instacnes_order = load(path2)
tuples = load(path3)

model = TruncatedSVD(n_components=k)
I = model.fit_transform(co_instance_pattern)

selected_instances = []
idx = np.argmax(I, axis=0)   # 获得每一列的最大值所在的行索引
for i in idx:
    selected_instances.append(order_instances[i]) 

    
    
    
    
    
    
    
    
    
    
    
print(selected_instances)
for term in selected_instances:
    print(tuples[term])


[('the executive committee', 'the president'), ('water', 'life'), ('the executive committee', 'the president'), ('protein', 'amino acid'), ('death', 'life')]
consist of
be a necessity for
consist of
be make up of
be the best part of


In [62]:
# 基于HITS算法实现

import os
import networkx as nx

def load(filename):
    with open(filename, 'rb') as output:
        data = pickle.load(output)
    return data

filename1 = 'co_instance_pattern.pkl' 
filename2 = 'instances_order.pkl'
filename3 = 'patterns_order.pkl'

path1 = os.path.join(os.getcwd(), filename1)
path2 = os.path.join(os.getcwd(), filename2)
path3 = os.path.join(os.getcwd(), filename3)

co_instance_pattern = load(path1)
instacnes_order = load(path2)
patterns_order = load(path3)

a,b = co_instance_pattern.shape
G = nx.Graph()
order_instances = {v:k for k,v in instances_order.items()}   # 反转原来的字典


for i in range(b):
    column = co_instance_pattern[:,i]
    ss = []
    for j in range(len(column)):
        if column[j] > 0:
            ss.append(j)
    print(ss, '----')
    for x in range(len(ss)):
        for y in range(x+1, len(ss)):
            G.add_edge(order_instances[x], order_instances[y])

h, a = nx.hits(G)

h = sorted(h.items(),key = lambda x:x[1],reverse = True)

print(h)

            

[0, 3, 15, 37, 60, 67, 102, 111, 113, 116, 119, 125, 129, 139, 147, 159, 185, 187, 199, 223, 238, 242, 245, 247, 253, 273, 286, 287, 303, 329, 342, 394, 400, 415, 426, 429, 443, 458, 526, 535, 550, 556, 571, 574, 612, 624, 625, 651, 677, 682, 687, 693, 700, 747, 768, 779, 817, 833, 844, 871, 879, 905, 1008, 1063, 1132, 1173, 1213, 1238, 1246, 1274, 1361, 1365, 1408, 1420, 1434, 1469, 1497, 1511, 1576, 1625, 1648, 1661, 1688, 1692, 1712, 1924, 1943, 2091, 2105, 2187, 2235, 2263, 2290, 2302, 2343, 2351, 2419, 2442, 2455, 2527, 2552, 2597, 2607] ----
[1, 10, 28, 39, 50, 62, 103, 122, 141, 212, 274, 300, 331, 339, 346, 356, 360, 375, 391, 392, 395, 427, 446, 453, 475, 478, 497, 543, 578, 584, 593, 607, 609, 629, 633, 661, 667, 669, 695, 710, 744, 750, 752, 776, 785, 825, 835, 857, 882, 886, 908, 972, 979, 1018, 1042, 1083, 1100, 1155, 1162, 1175, 1191, 1197, 1198, 1203, 1220, 1228, 1261, 1284, 1287, 1306, 1310, 1326, 1333, 1350, 1378, 1382, 1436, 1441, 1443, 1460, 1462, 1467, 1472, 1474, 1

[120, 127, 621] ----
[160, 217, 1777] ----
[2] ----
[38, 220, 724, 1778] ----
[7] ----
[207] ----
[231] ----
[35, 37, 232, 394, 429, 921, 1120, 1539, 1808] ----
[233, 618] ----
[235] ----
[237] ----
[8] ----
[2] ----
[240, 1107] ----
[64, 187, 244, 723, 1217] ----
[231, 247, 698, 1046] ----
[248] ----
[177, 250, 888, 1283, 1744] ----
[252] ----
[253, 347] ----
[254] ----
[8, 42, 56, 127, 258, 278, 328, 343, 362, 390, 524, 586, 862, 1184, 1241] ----
[259, 1254] ----
[261, 476, 927, 1528, 1764, 2205, 2214, 2570] ----
[262, 298] ----
[264, 383, 404, 740, 891, 1164, 1277, 1685, 1694, 1714, 2168, 2227, 2271, 2320, 2374] ----
[110, 147, 691, 1046] ----
[8, 758] ----
[8, 56, 266, 267, 617] ----
[8, 56] ----
[112, 181, 277, 407, 481, 784, 1158, 1224] ----
[280] ----
[282] ----
[2] ----
[290, 535, 964] ----
[100, 247, 592, 671, 819, 1097, 1211, 1758, 2071, 2197, 2213, 2256, 2399] ----
[2] ----
[294] ----
[298] ----
[63, 118, 220, 299, 606, 634, 721, 1181, 1480, 2591] ----
[8, 25, 34, 304] ----


[160, 1290] ----
[71, 630, 934, 1368, 1862] ----
[565] ----
[8, 56, 135, 577] ----
[758] ----
[34] ----
[8, 483, 673] ----
[328] ----
[1263] ----
[315] ----
[2] ----
[957] ----
[153] ----
[174] ----
[35, 235, 434, 1217] ----
[82] ----
[1272] ----
[1274] ----
[563] ----
[220, 1279] ----
[1280] ----
[527] ----
[551] ----
[187] ----
[328, 1312] ----
[717] ----
[160, 1302, 1702] ----
[85, 439] ----
[25] ----
[198, 1319] ----
[1321] ----
[500] ----
[668, 966, 1084, 1115, 2279] ----
[335] ----
[580] ----
[439] ----
[2] ----
[231, 334, 347, 518, 522, 1805] ----
[793, 1229] ----
[1336] ----
[156, 184] ----
[235] ----
[201] ----
[343, 393] ----
[0, 1278] ----
[448, 1341] ----
[1345] ----
[128] ----
[266] ----
[154, 1705] ----
[389, 500] ----
[306, 1071, 2218] ----
[1280] ----
[19] ----
[8, 1850] ----
[2] ----
[273, 308, 404, 925, 1798] ----
[1370] ----
[76] ----
[1373] ----
[44] ----
[65] ----
[1274] ----
[2] ----
[226] ----
[63, 1385] ----
[1387] ----
[1389] ----
[436] ----
[1392] ----
[1394] 

[(('protein', 'amino acid'), 0.0019011406844106464), (('vehicle', 'ukiah'), 0.0019011406844106464), (('the internet', 'information'), 0.0019011406844106464), (('a community', 'individual'), 0.0019011406844106464), (('symptom', 'high fever'), 0.0019011406844106464), (('book', 'a conclude chapter'), 0.0019011406844106464), (('the committee', 'five member'), 0.0019011406844106464), (('music', 'life'), 0.0019011406844106464), (('water', 'life'), 0.0019011406844106464), (('israel', 'the united nations'), 0.0019011406844106464), (('wellness center', 'kansas city'), 0.0019011406844106464), (('carbon', 'life'), 0.0019011406844106464), (('meal', 'early morning coffee'), 0.0019011406844106464), (('the fifth trunk', 'the fabric prop'), 0.0019011406844106464), (('six fruit basket', 'peach'), 0.0019011406844106464), (('the committee', 'eight member'), 0.0019011406844106464), (('the committee', 'twelve member'), 0.0019011406844106464), (('acupuncture', 'traditional chinese medicine'), 0.001901140684

In [59]:
d = {'a': 1, 'b': 4, 'c': 2, 'f' : 12}

a = sorted(d.items(), key=lambda x: x[1])


print(a)
print(a1)

[('a', 1), ('c', 2), ('b', 4), ('f', 12)]
[('f', 12), ('b', 4), ('c', 2), ('a', 1)]


In [61]:
print(co_instance_pattern)

[[15.  0.  0. ...  0.  0.  0.]
 [ 0.  1.  0. ...  0.  0.  0.]
 [ 0.  0.  9. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
